# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("homepage_actions.csv")
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [3]:
df.describe()

,id
count,8188.000000
mean,564699.749878
std,219085.845672
min,182988.000000
25%,373637.500000
50%,566840.500000
75%,758078.000000
max,937217.000000


In [4]:
# number of people in the experiment
df['group'].value_counts()

control       4264
experiment    3924
Name: group, dtype: int64

In [5]:
# Number of clicks and views
df['action'].value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [23]:
df['count'] = 1
group_exp= df[(df['group'] == 'experiment')].pivot(index = 'id',columns='action',values='count')
group_exp = group_exp.fillna(value = 0)
group_exp.head()

action,click,view
id,,
182988,0.0,1.0
183136,0.0,1.0
183141,1.0,1.0
183283,0.0,1.0
183389,0.0,1.0


In [24]:
group_exp.click.mean()

0.3097463284379172

In [26]:
group_ctlr = df[(df['group'] == 'control')].pivot(index='id', columns='action', values='count')
group_ctlr= group_ctlr.fillna(value=0)
group_ctlr['click'].mean()

0.2797118847539016

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [31]:
stats.ttest_ind(control.click,experiment.click,alternative='less')

Ttest_indResult(statistic=-2.6195696844542202, pvalue=0.0044125494574791466)

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [33]:
#Your code here
group_ctlr_rate = group_ctlr.mean()
new_group_exp = group_ctlr_rate * len(group_exp)
new_group_exp.click 

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [36]:
#Your code here
n = len(group_exp)
p = group_ctlr_rate
var = n * p * (1-p)
std = np.sqrt(var)
std

action
click    24.568548
view      0.000000
dtype: float64

In [37]:
#Your code here
actual_experiment_clicks = group_exp.click.sum()
z_score = (actual_experiment_clicks - new_group_exp )/std
print(z_score)

action
click    3.662536
view         -inf
dtype: float64


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [38]:
p_val = stats.norm.sf(z_score) #or 1 - stats.norm.cdf(z_score)
print(p_val)

[1.2486528e-04 1.0000000e+00]


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Though lower than the previously calculated value, the pvalue of 0.00012486 is still lower than the 0.05 alpha level suggesting rejection of the null hypothesis. This would mean that the
experimental homepage is more effective than the control homepage**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.